In [3]:
from environment import FinanceStock
from tradingbot import TradingBot,plot_treward,plot_performance,set_seeds
from backtester import TBBacktesterRM

In [16]:
symbols = ['2301.TW','2352.TW','2364.TW','2395.TW']
# ['1720.TW','3010.TW','2308.TW','2471.TW']
#['2207.TW','2376.TW','2377.TW','2314.TW']
# ['2317.TW','2382.TW','2891.TW','2886.TW']
# ['2347.TW','2353.TW','2356.TW','2357.TW']
# [**'2409.TW',**'2393.TW','2498.TW','2882.TW']
# ['2409.TW','2393.TW','3034.TW','3016.TW']
#['2330.TW','2379.TW','2412.TW','2881.TW','3008.TW','2303.TW','2344.TW','2449.TW']
features_fix = ['r','s','m','v']

In [5]:
class FinEnv:
    def __init__(self,symbols,features_fix,window=20,lags=3,leverage=1,episodes=61,split=0.6,valid=0.2,test=0.2):
        self.symbols = symbols
        self.features_fix = features_fix
        self.window = window
        self.lags = lags
        self.leverage = leverage
        self.episodes = episodes
        self.start = 0
        self.split = split
        self.valid = valid
        self.test = test 
        
        self.l_envs = dict()
        self.v_envs = dict()
        self.t_envs = dict()
        self.tradingbots = dict()
        self.backtesters = dict()
        
        self._build_env()
#         self._build_tradingbot()
        
    def _build_env(self):
        
        for symbol in self.symbols:
            learn_env = FinanceStock(symbol,[symbol] + self.features_fix,window=self.window,lags=self.lags,leverage=self.leverage,min_performance=0.9,
                         min_accuracy=0.475,start=self.start,end=self.split,mu=None,std=None)
            
            self.l_envs[symbol] = learn_env
    
            self.v_envs[symbol] = FinanceStock(symbol,features=learn_env.features,window=learn_env.window,lags=learn_env.lags,
                         leverage=learn_env.leverage,min_performance=0.0,min_accuracy=0.0,start=self.split,
                         end=self.split+self.valid,mu=learn_env.mu,std=learn_env.std)

            self.t_envs[symbol] = FinanceStock(symbol,features=learn_env.features,window=learn_env.window,lags=learn_env.lags,
                         leverage=learn_env.leverage,min_performance=0.0,min_accuracy=0.0,start=self.split+self.valid,
                         end=self.split+self.valid+self.test,mu=learn_env.mu,std=learn_env.std)
            
    
    def _build_tradingbot(self):
        
        for symbol in self.symbols:
            set_seeds(100)
            self.tradingbots[symbol] = TradingBot(self.l_envs[symbol],self.v_envs[symbol])
            self.tradingbots[symbol].learn(self.episodes)
            
            
    def _get_tradingbot(self):
        for symbol in self.symbols:
            set_seeds(100)
            self.tradingbots[symbol] = TradingBot(self.l_envs[symbol],self.v_envs[symbol])
#             self.tradingbots[symbol].learn(self.episodes)

In [17]:
env = FinEnv(symbols,features_fix)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## 未有model

In [18]:
env._build_tradingbot()

episode: 10/61 | treward:   16 | perf: 0.944 | av:  20.4 | max:  159 
 episode: 10/61 | VALIDATION | treward: 1865 | perf: 0.171 | eps: 0.92
episode: 20/61 | treward:  212 | perf: 0.877 | av: 316.7 | max: 3516 6 
 episode: 20/61 | VALIDATION | treward: 1865 | perf: 1.658 | eps: 0.83
episode: 30/61 | treward:  450 | perf: 0.887 | av: 751.4 | max: 3516 
 episode: 30/61 | VALIDATION | treward: 1865 | perf: 0.939 | eps: 0.75
episode: 40/61 | treward:   17 | perf: 0.924 | av: 896.2 | max: 3516  
 episode: 40/61 | VALIDATION | treward: 1865 | perf: 1.074 | eps: 0.68
episode: 50/61 | treward:   16 | perf: 1.047 | av: 1459.6 | max: 3516  
 episode: 50/61 | VALIDATION | treward: 1865 | perf: 1.287 | eps: 0.62
episode: 60/61 | treward:   57 | perf: 1.212 | av: 1590.7 | max: 3516  
 episode: 60/61 | VALIDATION | treward: 1865 | perf: 1.120 | eps: 0.56
INFO:tensorflow:Assets written to: ./models/2301.TW\assets| max: 3516 
episode: 10/61 | treward:   41 | perf: 1.060 | av:  28.4 | max:  304 
 episo

## 已有

In [ ]:
env._get_tradingbot()

In [ ]:
target = '2412.TW'

In [ ]:
env.tradingbots[target].model

In [ ]:
tb = TBBacktesterRM(env.t_envs[target],env.tradingbots[target].model,10000,0.0,0,verbose=True)

In [ ]:
tb.backtest_strategy(sl=0.015,tsl=None,tp=0.0185,wait=5)

https://havocfuture.tw/blog/python-indicators-talib